<a href="https://colab.research.google.com/github/Nirav-Madhani/CompilerConstrucction/blob/main/7_CC_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
inp = '''A -> B C
B -> a | €
C -> a b'''

In [ ]:
def inputToDict(inp):
  dt = dict()
  for l in inp.splitlines():
    key,value = l.split('->')
    key = key.strip()
    value = value.strip()
    dt[key] = [v.strip() for v in value.split('|')]
  return dt

In [ ]:
def dictToOutput(dic):
  ans = []
  for K,l in dic.items():
    v = ' | '.join(l)
    ans .append(' -> '.join([K,v]))
  return '\n'.join(ans)

#3 First Follow

In [ ]:
rules = inputToDict(inp)#{"A":["B C"],"B":["a","€"],"C":["a b"]}
rules

{'A': ['B C'], 'B': ['a', '€'], 'C': ['a b']}

In [ ]:
tokens = set()
for v in rules.values():
  for s in v:
    tokens.update(s.split(' '))
tokens

{'B', 'C', 'a', 'b', '€'}

In [ ]:
first = {}
def FindFirst(k):
  l = rules[k]
  for strs in l:
    tokens = strs.split(' ')
    for token in tokens:
      if token in rules.keys():
        temp = FindFirst(token)
        if k not in first.keys():
          first[k] = set()
        first[k].update(temp)                
      else:
        if k not in first.keys():
          first[k] = set()
        first[k].add(token)
        if '€' not in first[k]:
          break
  return first[k]
for k in rules.keys():
  FindFirst(k)
for t in tokens:
  if t not in first.keys():
    first[t]=set({t})
first

{'A': {'a', '€'},
 'B': {'a', '€'},
 'C': {'a'},
 'a': {'a'},
 'b': {'b'},
 '€': {'€'}}

In [ ]:
follow = {'A':set({'$'})}
def FindFollow(c):
  if c not in follow.keys():
    follow[c] = set()
  if c == '€': return
  for k,l in rules.items():
    for strs in l:
      tokens = strs.split(' ')
      for i,token in enumerate(tokens):        
        if token == c :
          if i != len(tokens)-1:
            follow[c].update(first[tokens[i+1]])
          else:
            follow[c].update(FindFollow(k))
  return follow[c]    
for t in rules.keys():  
  FindFollow(t)            
follow

{'A': {'$'}, 'B': {'a'}, 'C': {'$'}}

# 4 Left Recusion

In [ ]:
inp = '''A -> A B d | A a | a 
B -> B e | b'''
'''S -> A
A -> A d | A e | a B | a c
B -> b B c | f'''

'S -> A\nA -> A d | A e | a B | a c\nB -> b B c | f'

In [ ]:
rules = inputToDict(inp)
rules
tokens = set()
for v in rules.values():
  for s in v:
    tokens.update(s.split(' '))
tokens

{'A', 'B', 'a', 'b', 'd', 'e'}

In [ ]:
def removeUtil(rules):
  found = False
  rules_ = dict()
  for K,l in rules.items():
    alpha = []
    beta = []
    for v in l:
      v= v.strip()
      if v.split(' ')[0].strip() == K.strip():
        alpha.append(' '.join(v.split(' ')[1:]))
      else:
        beta.append(v)
    K_ = K+"'"
    #print(alpha,beta)
    if len(alpha) == 0:
      rules_[K] = rules[K]
      continue
    found = True
    if K not in rules_:
      rules_[K] = []
    for b in beta:
      rules_[K].append(' '.join([b,K_]))
    #print(rules_)
    if K_ not in rules_:
      rules_[K_] = []
    for a in alpha:
      rules_[K_].append(' '.join([a,K_]))
    rules_[K_].append('€')
    #print(rules_)
  if not found:
    return rules,False
  #rules_.update(rules)
  return rules_,found

def remove(rules):
  rules_,found = removeUtil(rules)
  print(rules_)
  if rules_ != rules:
    return remove(rules_)
  return rules_

In [ ]:
removeUtil(rules)

({'A': ["B A'", "c A'"], "A'": ["a A'"], 'B': ['c d']}, True)

In [ ]:
print(dictToOutput(remove(rules)))

{'A': ["a A'"], "A'": ["B d A'", "a A'", '€'], 'B': ["b B'"], "B'": ["e B'", '€']}
{'A': ["a A'"], "A'": ["B d A'", "a A'", '€'], 'B': ["b B'"], "B'": ["e B'", '€']}
A -> a A'
A' -> B d A' | a A' | €
B -> b B'
B' -> e B' | €
